# RewardFunction2
```
Key Elements:
1. Used waypoints to determine if track is straight or curved
   - Rewarded for increased speed if track is straight
   - If track is curved, increased speed is not rewarded
2. Check if all wheels on track
3. Determine progress and encourage completion of more steps (more iterations of the algorithm)
4. Be closer to the center of the track
```

In [2]:
def reward_function(params):
    '''
    Example of penalize steering, which helps mitigate zig-zag behaviors
    '''
   
    # Read input parameters
    #x_car = params['x']
    #y_car = params['y']
    on_track = params['all_wheels_on_track']
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    progress = params['progress']/100
    #waypoints = params['waypoints']
    #closest_waypoints = params['closest_waypoints']
    #steering = abs(params['steering_angle']) # Only need the absolute steering angle
    speed = params['speed']
    steps = params["steps"]
    
    REWARD_MIN = 0.1
    REWARD_MAX = 100
    TARGET_STEPS = 150
    
    # Calculate 3 marks that are farther and father away from the center line
    marker_1 = 0.25 * track_width
    marker_2 = 0.4 * track_width
    marker_3 = 0.6 * track_width
    
    # Set Base Reward
    if not on_track: # Fail them if off Track
        return REWARD_MIN
    elif progress == 1:
        #the lap is complete.  if we use more steps than TARGET_STEPS, the reward is lower
        return REWARD_MAX * TARGET_STEPS / steps
    elif progress < .1:
        #assuming that the car starts on a straight track, set the base reward to be the maximum
        #we will adjust things later on
        reward = REWARD_MAX
    elif steps > 0:        # we want the vehicle to continue making progress
        reward = REWARD_MAX * max(progress, 0.4)

    # Give higher reward if the car is closer to center line and vice versa
    if distance_from_center <= marker_1:
        reward += 5
    elif distance_from_center <= marker_2:
        reward += 1
    
    # Determine waypoints
    prev_pt = waypoints[closest_waypoints[0]] # closest waypoint behind car
    next_pt = waypoints[closest_waypoints[1]] # closest waypoint ahead of car
    nextnext_pt = waypoints[(closest_waypoints[1] + 1) % len(waypoints)] # waypoint after next_pt
    
    #car_next_dist = ((x_car - next_pt[0])**2 + (y_car - next_pt[1])**2)**0.5    
    prev_next_dist = ((prev_pt[0] - next_pt[0])**2 + (prev_pt[1] - next_pt[1])**2)**0.5    
    next_nextnext_dist = ((next_pt[0] - nextnext_pt[0])**2 + (next_pt[1] - nextnext_pt[1])**2)**0.5   
    #car_nextnext_dist = ((x_car - nextnext_pt[0])**2 + (y_car - nextnext_pt[1])**2)**0.5
    prev_nextnext_dist = ((prev_pt[0] - nextnext_pt[0])**2 + (prev_pt[1] - nextnext_pt[1])**2)**0.5
    
    # Find out whether track is straight or curved
    if prev_nextnext_dist/(prev_next_dist + next_nextnext_dist) > .98:
        # reward speed on straight track
        reward += min(speed**2, 10)
    elif speed > 2:
        #reward speed on curved track
        reward += 2
 
    return float(reward)
